In [9]:
import pandas as pd
from sklearn import datasets
import numpy as np
import random

iris = datasets.load_iris()
irisDF = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
irisDF = irisDF.sample(frac=1,random_state=1).reset_index(drop=True)

print(irisDF)

trainIris = irisDF[:110]

testIris = irisDF[110:].reset_index(drop=True)
testIris['hyp'] = np.nan


     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                  5.8               4.0                1.2               0.2   
1                  5.1               2.5                3.0               1.1   
2                  6.6               3.0                4.4               1.4   
3                  5.4               3.9                1.3               0.4   
4                  7.9               3.8                6.4               2.0   
..                 ...               ...                ...               ...   
145                6.3               2.8                5.1               1.5   
146                6.4               3.1                5.5               1.8   
147                6.3               2.5                4.9               1.5   
148                6.7               3.1                5.6               2.4   
149                4.9               3.6                1.4               0.1   

     target  
0       0.0  

In [10]:
def trainZeroR(train):
    return np.argmax(np.bincount(train.target))
    
def testIrisZeroR(test, model):
    for i in test.index:
        test.hyp[i] = model

In [89]:
model = trainZeroR(trainIris)

testIrisZeroR(testIris, model)

print(testIris)

    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                 5.6               2.8                4.9               2.0   
1                 6.8               3.2                5.9               2.3   
2                 5.0               3.3                1.4               0.2   
3                 5.1               3.7                1.5               0.4   
4                 5.9               3.2                4.8               1.8   
5                 4.6               3.1                1.5               0.2   
6                 5.8               2.7                5.1               1.9   
7                 4.8               3.1                1.6               0.2   
8                 6.5               3.0                5.2               2.0   
9                 4.9               2.5                4.5               1.7   
10                4.6               3.2                1.4               0.2   
11                6.4               3.2 

In [13]:
def discretize(data, means, featuresToDiscretize):
    
    dataDiscret = data.copy()
    
    for col in featuresToDiscretize:
        for i in data.index:
            
            if (data[col][i] >= means[col]):
                dataDiscret[col][i] = "long"
            else:
                dataDiscret[col][i] = "court"
                
    return dataDiscret


trainIrisDis = discretize(trainIris, trainIris.mean(), iris['feature_names'])
testIrisDis = discretize(testIris, trainIris.mean(), iris['feature_names'])


print(testIrisDis)

   sepal length (cm) sepal width (cm) petal length (cm) petal width (cm)  \
0              court            court              long             long   
1               long             long              long             long   
2              court             long             court            court   
3              court             long             court            court   
4              court             long              long             long   
5              court             long             court            court   
6              court            court              long             long   
7              court             long             court            court   
8               long            court              long             long   
9              court            court              long             long   
10             court             long             court            court   
11              long             long              long             long   
12          

C:\Users\romai\AppData\Local\Temp\ipykernel_9292\1950391893.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataDiscret[col][i] = "long"
C:\Users\romai\AppData\Local\Temp\ipykernel_9292\1950391893.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataDiscret[col][i] = "court"


In [96]:
def trainOneR(train, featuresColumns):
    
    bestPrecision = -1

    for col in featuresColumns:
        res = pd.DataFrame(data = np.zeros((len(train.target.unique()), len(train[col].unique()))), columns = train[col].unique())

        for i in train.index:
            res[train[col][i]][int(train.target[i])] += 1
        
        precision = res.max().sum() / res.sum().sum()

        #Update du model avec nouvelle colone
        if (precision > bestPrecision):
            bestPrecision = precision
            model = {'feature_name': col}
            
            for value in res:
                print()
                model[value] = res.idxmax()[value]

    return model



In [101]:
def testOneR(test, model):
    for i in test.index:
        test.hyp[i] = model[ test[ model['feature_name'] ][i] ]

model = trainOneR(trainIrisDis, iris["feature_names"])
print(model)
    
testOneR(testIrisDis, model)
testIrisDis.hyp = np.nan

testOneR(testIrisDis, model)
print(testIrisDis.hyp)






{'feature_name': 'petal length (cm)', 'court': 0, 'long': 2}
0     2.0
1     2.0
2     0.0
3     0.0
4     2.0
5     0.0
6     2.0
7     0.0
8     2.0
9     2.0
10    0.0
11    2.0
12    0.0
13    2.0
14    0.0
15    0.0
16    0.0
17    0.0
18    0.0
19    0.0
20    0.0
21    2.0
22    2.0
23    0.0
24    2.0
25    2.0
26    2.0
27    2.0
28    2.0
29    0.0
30    0.0
31    2.0
32    2.0
33    2.0
34    0.0
35    2.0
36    2.0
37    2.0
38    2.0
39    0.0
Name: hyp, dtype: float64


C:\Users\romai\AppData\Local\Temp\ipykernel_9292\2513807878.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test.hyp[i] = model[ test[ model['feature_name'] ][i] ]
